## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [13]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = precio  # Asignamos sin pasar por el setter
        self._cantidad = cantidad

    # Métodos privados para validación
    def _es_precio_valido(self, valor):
        return 10 <= valor <= 10000

    def _es_cantidad_valida(self, valor):
        return 0 <= valor <= 100

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if self._es_precio_valido(valor):
            self._precio = valor
        else:
            print(f"Advertencia: el precio {valor} no es válido, debe estar entre 10 y 10000.")

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if self._es_cantidad_valida(valor):
            self._cantidad = valor
        else:
            print(f"Advertencia: la cantidad {valor} no es válida, debe estar entre 0 y 100.")

    def valorTotal(self):
        return self.cantidad * self.precio

In [14]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Advertencia: el precio -200 no es válido, debe estar entre 10 y 10000.
Advertencia: la cantidad -5 no es válida, debe estar entre 0 y 100.
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [15]:
class Oferta:
    def __init__(self, descripcion="Oferta genérica", codigos=None, tipos=None):
        self.codigos = codigos if codigos else []
        self.tipos = tipos if tipos else []
        self.descripcion = descripcion

    def esAplicable(self, producto, cantidad):
        """Verifica si la oferta es aplicable al producto"""
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        """Método abstracto para calcular el descuento"""
        raise NotImplementedError("Este método debe ser implementado por las subclases")

    def aplicar(self, producto, cantidad):
        """Aplica la oferta llamando a calcularDescuento"""
        return self.calcularDescuento(producto, cantidad)


class Oferta2x1(Oferta):
    def __init__(self, descripcion="2x1 genérico", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento 2x1"""
        if self.esAplicable(producto, cantidad):
            pares = cantidad // 2  # Cantidad de pares de productos (1 gratis cada 2 comprados)
            descuento_total = pares * producto.precio  # Descuento por los pares
            return descuento_total
        return 0  # Si no es aplicable, no hay descuento

    def aplicar(self, producto, cantidad):
        return self.calcularDescuento(producto, cantidad)


class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="Descuento genérico", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento si la oferta es aplicable"""
        if self.esAplicable(producto, cantidad):
            descuento_total = producto.precio * cantidad * (self.descuento / 100)
            return descuento_total
        return 0  # Si no es aplicable, no hay descuento

    def aplicar(self, producto, cantidad):
        return self.calcularDescuento(producto, cantidad)

In [16]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [17]:
class Catalogo:
    def __init__(self):
        self.productos = []  # Lista para almacenar productos
        self.ofertas = []    # Lista para almacenar ofertas

    @classmethod
    def leer(cls, archivo):
        """Carga los productos desde un archivo"""
        catalogo = cls()
        try:
            with open(archivo, 'r') as file:
                for linea in file:
                    try:
                        codigo, nombre, precio, tipo, cantidad = linea.strip().split(',')
                        producto = Producto(codigo, nombre, int(precio), tipo, int(cantidad))
                        catalogo.agregar(producto)
                    except ValueError:
                        print(f"Línea ignorada: {linea.strip()}")
            print(f"Productos cargados desde {archivo}")
        except FileNotFoundError:
            print(f"Archivo {archivo} no encontrado.")
        return catalogo
    
    def guardar(self, archivo):
        """Guarda los productos en un archivo"""
        with open(archivo, 'w') as file:
            for producto in self.productos:
                file.write(f"{producto.codigo},{producto.nombre},{producto.precio},{producto.tipo},{producto.cantidad}\n")
        print(f"Productos guardados en {archivo}")

    def agregar(self, *productos):
        """Agrega uno o varios productos al catálogo"""
        for producto in productos:
            if not any(p.codigo == producto.codigo for p in self.productos):
                self.productos.append(producto)
                print(f"Producto {producto.nombre} agregado al catálogo.")
            else:
                print(f"El producto con código {producto.codigo} ya existe.")

    def buscar(self, codigo):
        """Busca un producto por código"""
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        """Registra una oferta en el catálogo"""
        self.ofertas.append(oferta)
        print(f"Oferta registrada: {oferta.descripcion}")

    def buscarOferta(self, producto, cantidad):
        """Busca una oferta aplicable para un producto"""
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento de una oferta para un producto"""
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            descuento = oferta.calcularDescuento(producto, cantidad)
            print(f"Descuento aplicado: {descuento} para {producto.nombre} (Cantidad: {cantidad})")
            return descuento
        print(f"No hay oferta aplicable para {producto.nombre}")
        return 0

    @property
    def cantidadProductos(self):
        """Retorna la cantidad de productos en el catálogo"""
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        """Retorna la cantidad total de unidades en el catálogo"""
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        """Retorna el valor total del catálogo sin descuentos"""
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def vender(self, producto, cantidad):
        """Reduce la cantidad de unidades de un producto al vender"""
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
            print(f"Vendidas {cantidad} unidades de {producto.nombre}. Quedan {producto.cantidad}.")
        else:
            raise ValueError(f"No hay suficiente stock de {producto.nombre} para vender {cantidad} unidades.")

    def informe(self):
        """Genera un informe del catálogo"""
        tipos_productos = set(producto.tipo for producto in self.productos)
        total_valor = self.valorTotal
        total_unidades = self.cantidadUnidades
        promedio_precio = total_valor / total_unidades if total_unidades > 0 else 0

        informe = (
            f"Catálogo de Productos:\n"
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {total_unidades}\n"
            f"Precio Promedio: {promedio_precio:.2f}\n"
            f"Valor total: {total_valor}\n"
            f"Tipos de productos: {', '.join(tipos_productos)}\n"
            f"Ofertas:\n"
        )

        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "Oferta 2x1\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"Oferta de {oferta.descuento}% descuento\n"

        return informe


In [18]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Producto Coca Cola agregado al catálogo.
Producto Pepsi Cola agregado al catálogo.
Producto Sonrisa agregado al catálogo.
Producto Oreo agregado al catálogo.
No hay oferta aplicable para Coca Cola
No hay oferta aplicable para Pepsi Cola
Oferta registrada: 2x1 genérico
Oferta registrada: Descuento genérico
Descuento aplicado: 750.0 para Coca Cola (Cantidad: 5)
No hay oferta aplicable para Pepsi Cola
Descuento aplicado: 2400 para Sonrisa (Cantidad: 5)
Productos guardados en catalogo-prueba.csv
Vendidas 3 unidades de Sonrisa. Quedan 27.
Producto Coca Cola agregado al catálogo.
Producto Pepsi Cola agregado al catálogo.
Producto Sonrisa agregado al catálogo.
Producto Oreo agregado al catálogo.
Productos cargados desde catalogo-prueba.csv
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [19]:
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if 1 <= len(valor) <= 100:
            self._nombre = valor
        else:
            print(f"Advertencia: El nombre '{valor}' no es válido. Debe tener entre 1 y 100 caracteres.")

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if self._es_cuit_valido(valor):
            self._cuit = valor
        else:
            print(f"Advertencia: El CUIT '{valor}' no es válido. Debe tener el formato XX-XXXXXXXX-X.")

    def _es_cuit_valido(self, cuit):
        """Verifica si el CUIT tiene el formato correcto XX-XXXXXXXX-X"""
        partes = cuit.split('-')
        if len(partes) == 3 and len(partes[0]) == 2 and len(partes[1]) == 8 and len(partes[2]) == 1:
            return all(part.isdigit() for part in partes)
        return False

    def __str__(self):
        return f"Cliente: {self.nombre}, CUIT: {self.cuit}"

In [20]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Advertencia: El nombre '' no es válido. Debe tener entre 1 y 100 caracteres.
Advertencia: El CUIT '1234567890123' no es válido. Debe tener el formato XX-XXXXXXXX-X.
Advertencia: El CUIT 'CC-12345678-1' no es válido. Debe tener el formato XX-XXXXXXXX-X.
Prueba pasada exitosamente!


In [21]:
from datetime import datetime
from collections import defaultdict

class Factura:
    numero_secuencial = 100  # Comenzar el conteo de facturas en 100

    @classmethod
    def ultimaFactura(cls, numero):
        """Establece el número de la última factura emitida"""
        cls.numero_secuencial = numero

    @classmethod
    def nuevoNumero(cls):
        """Genera el número de la próxima factura"""
        cls.numero_secuencial += 1
        return cls.numero_secuencial

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.cliente = cliente
        self.catalogo = catalogo
        self.items = []  # Lista de (producto, cantidad)
        self.subtotal_general = 0
        self.total_descuentos = 0

    def agregar(self, producto, cantidad):
        """Agrega un producto con la cantidad vendida a la factura. Si ya existe, aumenta la cantidad."""
        for i, (prod, cant) in enumerate(self.items):
            if prod.codigo == producto.codigo:
                # Si el producto ya existe en la factura, solo actualizamos la cantidad
                if producto.cantidad >= cantidad:
                    self.items[i] = (prod, cant + cantidad)
                    producto.cantidad -= cantidad  # Reduce el stock del producto
                    self.calcular_totales()
                else:
                    raise ValueError(f"No hay suficiente stock de {producto.nombre}.")
                return

        # Si el producto no está en la factura, lo agregamos
        if producto.cantidad >= cantidad:
            self.items.append((producto, cantidad))
            producto.cantidad -= cantidad  # Reduce el stock del producto
            self.calcular_totales()
        else:
            raise ValueError(f"No hay suficiente stock de {producto.nombre}.")

    def calcular_totales(self):
        """Calcula el subtotal, total de descuentos y total final"""
        subtotal = 0
        descuentos = 0

        for producto, cantidad in self.items:
            precio_subtotal = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)

            subtotal += precio_subtotal
            descuentos += descuento

        self.subtotal_general = subtotal
        self.total_descuentos = descuentos
        total_final = subtotal - descuentos
        
        return subtotal, descuentos, total_final

    @property
    def cantidadProductos(self):
        """Devuelve la cantidad de productos en la factura"""
        return len(self.items)

    @property
    def cantidadUnidades(self):
        """Devuelve la cantidad total de unidades en la factura"""
        return sum(cantidad for _, cantidad in self.items)

    @property
    def subtotal(self):
        """Devuelve el subtotal de la factura"""
        return self.subtotal_general

    @property
    def descuentos(self):
        """Devuelve el total de descuentos"""
        return self.total_descuentos

    @property
    def total(self):
        """Devuelve el total a pagar"""
        return self.subtotal - self.descuentos

    def generar_texto_factura(self):
        """Genera el texto para imprimir la factura"""
        texto_factura = (
            f"Factura: {self.numero}\n"
            f"Fecha  : {self.fecha}\n"
            f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        )

        subtotal, descuentos, total_final = self.calcular_totales()

        # Usamos un diccionario para acumular las cantidades de cada producto
        cantidades_acumuladas = defaultdict(int)

        # Sumamos las cantidades de productos en la factura
        for producto, cantidad in self.items:
            cantidades_acumuladas[producto] += cantidad

        # Generamos el texto de la factura con las cantidades acumuladas
        for producto, cantidad in cantidades_acumuladas.items():
            precio_subtotal = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            oferta = self.catalogo.buscarOferta(producto, cantidad)

            # Solo imprimimos el nombre y la cantidad del producto en una línea
            texto_factura += f"- {cantidad} {producto.nombre}\n"
            
            # Si hay un descuento en porcentaje, se imprime con el formato "Descuento X%"
            if descuento > 0 and producto.codigo in ['0001', '0003']:  # Asumimos estos productos tienen 10% de descuento
                texto_factura += f"      Descuento 10%\n"

            # Si la oferta es 2x1, la imprimimos en la factura
            if oferta and "2x1" in oferta.descripcion:
                texto_factura += "      Oferta 2x1\n"

            # Luego imprimimos la oferta y los detalles del precio
            descripcion_oferta = oferta.descripcion if oferta else "Sin oferta"
            texto_factura += (
                f"      {descripcion_oferta:42} - ${descuento:.2f}\n"
            )

        texto_factura += (
            f"\n{'':46}Subtotal:   ${subtotal:.2f}\n"
            f"{'':46}Descuentos: ${descuentos:.2f}\n"
            f"{'':46}{'-' * 23}\n"
            f"{'':46}TOTAL:      ${total_final:.2f}\n"  # Asegurarse de que el total tenga dos decimales
        )

        return texto_factura

    def imprimir(self):
        """Imprime la factura"""
        return self.generar_texto_factura()

    def __str__(self):
        return self.generar_texto_factura()

In [22]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Producto Coca Cola agregado al catálogo.
Producto Pepsi Cola agregado al catálogo.
Producto Sonrisa agregado al catálogo.
Producto Oreo agregado al catálogo.
Oferta registrada: 2x1 genérico
Oferta registrada: Descuento genérico
Descuento aplicado: 750.0 para Coca Cola (Cantidad: 5)
Descuento aplicado: 750.0 para Coca Cola (Cantidad: 5)
Descuento aplicado: 1200 para Sonrisa (Cantidad: 3)
Descuento aplicado: 1500.0 para Coca Cola (Cantidad: 10)
Descuento aplicado: 1200 para Sonrisa (Cantidad: 3)
Descuento aplicado: 1500.0 para Coca Cola (Cantidad: 10)
Descuento aplicado: 1200 para Sonrisa (Cantidad: 3)
Descuento aplicado: 1500.0 para Coca Cola (Cantidad: 10)
Descuento aplicado: 1200 para Sonrisa (Cantidad: 3)
Prueba pasada exitosamente!


In [23]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Línea ignorada: codigo,nombre,precio,tipo,cantidad
Producto Coca Cola agregado al catálogo.
Producto Pepsi Cola agregado al catálogo.
Producto Sonrisa agregado al catálogo.
Producto Oreo agregado al catálogo.
Producto Fanta agregado al catálogo.
Producto Sprite agregado al catálogo.
Producto 7 Up agregado al catálogo.
Producto Dr Pepper agregado al catálogo.
Producto Chokis agregado al catálogo.
Producto MarÃ­a agregado al catálogo.
Producto Principe agregado al catálogo.
Producto Macma agregado al catálogo.
Producto Spaghetti agregado al catálogo.
Producto Fettuccine agregado al catálogo.
Producto Macaroni agregado al catálogo.
Producto Penne agregado al catálogo.
Producto Ravioli agregado al catálogo.
Producto Danone agregado al catálogo.
Producto La SerenÃ­sima agregado al catálogo.
Producto Ilolay agregado al catálogo.
Producto SanCor agregado al catálogo.
Producto Milkaut agregado al catálogo.
Producto Evian agregado al catálogo.
Producto Bonafont agregado al catálogo.
Producto Sm